In [1]:
import requests
import json 
import pickle
import os 
import pandas as pd 
import json
import time
import math 
import numpy as np

In [2]:
# IMPORTANT: ne jamais versionner de clés API.
# CHANGE: clés API en dur -> variables d'environnement (évite de commit des secrets).
# Les clés sont lues depuis des variables d'environnement (fichier .env ignoré par Git).

headers_myapp = {
    'API-Key': os.getenv('AFKLM_API_KEY_MYAPP', '<REDACTED>'),
    'Accept': 'application/hal+json'
}
headers_myapp2 = {
    'API-Key': os.getenv('AFKLM_API_KEY_MYAPP2', '<REDACTED>'),
    'Accept': 'application/hal+json'
}
headers_extract1 = {
    'API-Key': os.getenv('AFKLM_API_KEY_EXTRACT1', '<REDACTED>'),
    'Accept': 'application/hal+json'
}

Set up extraction follow up file 

In [ ]:
# Appels pour avoir le nombre de pages par jour sur le mois de janvier
import datetime
startrange_ = datetime.datetime(2026, 1, 1)
numdays = 32
date_list = [startrange_ + datetime.timedelta(days=x) for x in range(numdays)]
date_list = [sub.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for sub in date_list]

totalPagesPerDay_ = []
for i in range(len(date_list)-1):
    #if i not in [0,1]:
        try: 
            print(date_list[i])
            print(date_list[i+1])

            startrange_ = date_list[i]
            endrange_ = date_list[i+1]
            #endrange_ = "2026-01-01T08%3A00%3A00.000Z"

            # Premier appel pour avoir le nombre de pages 
            response = requests.get(url=f"https://api.airfranceklm.com/opendata/flightstatus?startRange={startrange_}&endRange={endrange_}", headers=headers)
            print(response.status_code)
            totalPages = json.loads(response.content.decode('utf-8'))["page"]['totalPages'] # Nombre total de pages 
            totalPagesPerDay_.append(totalPages)
            time.sleep(5)
        except:
            print(f"Call {startrange_} to {endrange_} failed with error {response.status_code}")
            totalPagesPerDay_.append("failed")
            time.sleep(5)

# Create DF
totalPagesPerDay_df = pd.DataFrame({"from" : date_list[0:len(date_list)-1],
                                    "to" : date_list[1:len(date_list)],
                                    "totalPages" : totalPagesPerDay_}
                                    )

# Add necessary columns for extraction 
totalPagesPerDay_df['lastExtractedPage'] = np.nan
worker_array = ["myapp","myapp2","extract1"]
totalPagesPerDay_df['worker'] = np.repeat(worker_array, math.ceil(totalPagesPerDay_df.shape[0]/3))[0:totalPagesPerDay_df.shape[0]]
totalPagesPerDay_df['complete'] = np.repeat(False, totalPagesPerDay_df.shape[0])

# Save DF
totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
sum(totalPagesPerDay_df.loc[totalPagesPerDay_df.totalPages != "failed"]["totalPages"].apply(lambda x:int(x))) * 100  # taille espérée de la base de données 


In [6]:
totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
totalPagesPerDay_df.head()

,from,to,totalPages,lastExtractedPage,worker,complete
0,2026-01-01T00:00:00.000000Z,2026-01-02T00:00:00.000000Z,87,87.0,myapp,True
1,2026-01-02T00:00:00.000000Z,2026-01-03T00:00:00.000000Z,94,94.0,myapp,True
2,2026-01-03T00:00:00.000000Z,2026-01-04T00:00:00.000000Z,failed,NaN,myapp,False
3,2026-01-04T00:00:00.000000Z,2026-01-05T00:00:00.000000Z,failed,NaN,myapp,False
4,2026-01-05T00:00:00.000000Z,2026-01-06T00:00:00.000000Z,failed,NaN,myapp,False


Set up extraction

In [4]:
try:
    os.mkdir('/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/data/afklm_large/')
except:
    print("folder exists")

folder exists


In [7]:
headers_attribution = {"myapp":headers_myapp,
                       "myapp2":headers_myapp2,
                       "extract1":headers_extract1}


for worker_ in set(totalPagesPerDay_df['worker']):
    print(worker_)
    # Remaining work 
    sub_df = totalPagesPerDay_df.loc[(totalPagesPerDay_df['worker'] == worker_) & (-totalPagesPerDay_df['complete']) & (totalPagesPerDay_df['totalPages']!='failed')]
    #  
    for row_ in range(sub_df.shape[0]):
        from_, to_, tp_, lep_ = sub_df.iloc[row_][["from", "to","totalPages", "lastExtractedPage"]]
            
        print(f"{from_} commence.")

        if math.isnan(lep_):
            start_page_ = 0
        else:
            start_page_ = int(lep_) + 1 
        # Appel
        for page_ in range(start_page_, int(tp_)+1):
            try:
                response = requests.get(url=f"https://api.airfranceklm.com/opendata/flightstatus?startRange={from_}&endRange={to_}&pageNumber={page_}", headers=headers_attribution[worker_])
                with open(f'/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/data/afklm_large/AFKLM_get_flight_{from_[0:10].replace("-","_")}_{to_[0:10].replace("-","_")}_page_{page_}.json', 'w') as f:
                    json.dump(response.json(), f, indent=4)
                
            except:
                print(f"Call page {page_} failed with error {response.status_code} --> Nombre de calls atteint pour ce worker.")
                if response.status_code == 403:
                    totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
                    if start_page_ == page_:
                        if start_page_ == 0 :
                            lep_update = np.nan
                        else:
                            lep_update = page_ - 1
                    else:
                        lep_update = page_ - 1 
                    totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,'lastExtractedPage']= lep_update
                    totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
                    break


            if page_ == int(tp_):
                totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
                totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,"lastExtractedPage"] = page_
                totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,"complete"] = True
                totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
                print(f"{from_} extrait.")
            time.sleep(5)




myapp
2026-01-07T00:00:00.000000Z
Call page 40 failed with error 504
2026-01-07T00:00:00.000000Z extrait.
2026-01-08T00:00:00.000000Z
Call page 29 failed with error 403
2026-01-09T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-10T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-11T00:00:00.000000Z
Call page 0 failed with error 403
myapp2
2026-01-14T00:00:00.000000Z
Call page 72 failed with error 500
Call page 88 failed with error 500
2026-01-14T00:00:00.000000Z extrait.
2026-01-15T00:00:00.000000Z
Call page 1 failed with error 500
Call page 6 failed with error 403
2026-01-17T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-18T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-19T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-21T00:00:00.000000Z
Call page 0 failed with error 403
2026-01-22T00:00:00.000000Z
Call page 0 failed with error 403
extract1
2026-01-29T00:00:00.000000Z
Call page 55 failed with error 500
2026-01-29T00:00:00.00000

In [5]:
## Get missing pages

dir_ = '/home/pierre/Documents/DE_DATASCIENTEST/data_projet_dst_airlines/afklm_large/'
def get_missing_pages(monitoring_file, data_dir):

    for f_ in monitoring_file["from"]:
        print(f_)
        tp_ = monitoring_file.loc[monitoring_file['from']==f_,"totalPages"].iloc[0]
        if (tp_ == 'failed'):
            print("No data for this day")
        else:
            p_range = list(range(0, int(tp_)+1))
            files_ = os.listdir(data_dir)
            files_ = list(filter(lambda x: x[17:27].replace("_","-") == f_[0:10], files_))
            pages_ = [parse_file_name(x) for x in files_]
            #print(p_range)
            #print(pages_)
            print(list(filter(lambda x: x not in pages_, p_range)))
        

def parse_file_name(file,):
    from_ = file[17:27]
    to_ = file[28:38]
    page_ =  file.replace(f"AFKLM_get_flight_{from_}_{to_}_page_","")
    page_ =  page_.replace(".json","")
    return int(page_)


get_missing_pages(totalPagesPerDay_df, dir_)

2026-01-01T00:00:00.000000Z
[0, 1, 3, 43]
2026-01-02T00:00:00.000000Z
[0, 1, 44, 73, 79, 80]
2026-01-03T00:00:00.000000Z
No data for this day
2026-01-04T00:00:00.000000Z
No data for this day
2026-01-05T00:00:00.000000Z
No data for this day
2026-01-06T00:00:00.000000Z
No data for this day
2026-01-07T00:00:00.000000Z
[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
2026-01-08T00:00:00.000000Z
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91]
2026-01-09T